In [ ]:
class Tweet:

    def __init__(self):
        pass

In [ ]:
class TweetCriteria:
	
	def __init__(self):
		self.maxTweets = 0
		self.within = "15mi"
		
	def setUsername(self, username):
		self.username = username
		return self
		
	def setSince(self, since):
		self.since = since
		return self
	
	def setUntil(self, until):
		self.until = until
		return self
		
	def setQuerySearch(self, querySearch):
		self.querySearch = querySearch
		return self
		
	def setMaxTweets(self, maxTweets):
		self.maxTweets = maxTweets
		return self

	def setTopTweets(self, topTweets):
		self.topTweets = topTweets
		return self
	
	def setNear(self, near):
		self.near = near
		return self

	def setWithin(self, within):
		self.within = within
		return self

In [ ]:
import urllib,urllib2,json,re,datetime,sys,cookielib

from pyquery import PyQuery

class TweetManager:
	
	def __init__(self):
		pass
		
	@staticmethod
	def getTweets(tweetCriteria, receiveBuffer=None, bufferLength=100, proxy=None):
		refreshCursor = ''
	
		results = []
		resultsAux = []
		cookieJar = cookielib.CookieJar()
		
		if hasattr(tweetCriteria, 'username') and (tweetCriteria.username.startswith("\'") or tweetCriteria.username.startswith("\"")) and (tweetCriteria.username.endswith("\'") or tweetCriteria.username.endswith("\"")):
			tweetCriteria.username = tweetCriteria.username[1:-1]

		active = True

		while active:
			json = TweetManager.getJsonReponse(tweetCriteria, refreshCursor, cookieJar, proxy)
			if len(json['items_html'].strip()) == 0:
				break

			refreshCursor = json['min_position']
			scrapedTweets = PyQuery(json['items_html'])
			#Remove incomplete tweets withheld by Twitter Guidelines
			scrapedTweets.remove('div.withheld-tweet')
			tweets = scrapedTweets('div.js-stream-tweet')
			
			if len(tweets) == 0:
				break
			
			for tweetHTML in tweets:
				tweetPQ = PyQuery(tweetHTML)
				tweet = Tweet()
				
				usernameTweet = tweetPQ("span:first.username.u-dir b").text()
				txt = re.sub(r"\s+", " ", tweetPQ("p.js-tweet-text").text().replace('# ', '#').replace('@ ', '@'))
				retweets = int(tweetPQ("span.ProfileTweet-action--retweet span.ProfileTweet-actionCount").attr("data-tweet-stat-count").replace(",", ""))
				favorites = int(tweetPQ("span.ProfileTweet-action--favorite span.ProfileTweet-actionCount").attr("data-tweet-stat-count").replace(",", ""))
				dateSec = int(tweetPQ("small.time span.js-short-timestamp").attr("data-time"))
				id = tweetPQ.attr("data-tweet-id")
				permalink = tweetPQ.attr("data-permalink-path")
				
				geo = ''
				geoSpan = tweetPQ('span.Tweet-geo')
				if len(geoSpan) > 0:
					geo = geoSpan.attr('title')
				
				tweet.id = id
				tweet.permalink = 'https://twitter.com' + permalink
				tweet.username = usernameTweet
				tweet.text = txt
				tweet.date = datetime.datetime.fromtimestamp(dateSec)
				tweet.retweets = retweets
				tweet.favorites = favorites
				tweet.mentions = " ".join(re.compile('(@\\w*)').findall(tweet.text))
				tweet.hashtags = " ".join(re.compile('(#\\w*)').findall(tweet.text))
				tweet.geo = geo
				
				results.append(tweet)
				resultsAux.append(tweet)
				
				if receiveBuffer and len(resultsAux) >= bufferLength:
					receiveBuffer(resultsAux)
					resultsAux = []
				
				if tweetCriteria.maxTweets > 0 and len(results) >= tweetCriteria.maxTweets:
					active = False
					break
					
		
		if receiveBuffer and len(resultsAux) > 0:
			receiveBuffer(resultsAux)
		
		return results
	
	@staticmethod
	def getJsonReponse(tweetCriteria, refreshCursor, cookieJar, proxy):
		url = "https://twitter.com/i/search/timeline?f=tweets&q=%s&src=typd&max_position=%s"

		urlGetData = ''
		
		if hasattr(tweetCriteria, 'username'):
			urlGetData += ' from:' + tweetCriteria.username
		
		if hasattr(tweetCriteria, 'querySearch'):
			urlGetData += ' ' + tweetCriteria.querySearch
		
		if hasattr(tweetCriteria, 'near'):
			urlGetData += "&near:" + tweetCriteria.near + " within:" + tweetCriteria.within
		
		if hasattr(tweetCriteria, 'since'):
			urlGetData += ' since:' + tweetCriteria.since
			
		if hasattr(tweetCriteria, 'until'):
			urlGetData += ' until:' + tweetCriteria.until
		

		if hasattr(tweetCriteria, 'topTweets'):
			if tweetCriteria.topTweets:
				url = "https://twitter.com/i/search/timeline?q=%s&src=typd&max_position=%s"
		
		
		url = url % (urllib.quote(urlGetData), refreshCursor)

		headers = [
			('Host', "twitter.com"),
			('User-Agent', "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36"),
			('Accept', "application/json, text/javascript, */*; q=0.01"),
			('Accept-Language', "de,en-US;q=0.7,en;q=0.3"),
			('X-Requested-With', "XMLHttpRequest"),
			('Referer', url),
			('Connection', "keep-alive")
		]

		if proxy:
			opener = urllib2.build_opener(urllib2.ProxyHandler({'http': proxy, 'https': proxy}), urllib2.HTTPCookieProcessor(cookieJar))
		else:
			opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookieJar))
		opener.addheaders = headers

		try:
			response = opener.open(url)
			jsonResponse = response.read()
		except:
			print "Twitter weird response. Try to see on browser: https://twitter.com/search?q=%s&src=typd" % urllib.quote(urlGetData)
			sys.exit()
			return
		
		dataJson = json.loads(jsonResponse)
		
		return dataJson		

In [19]:
import csv
import pandas as pd
with open('CU_segment_sentiment_data_Prayash.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    df = pd.DataFrame([])
    for row in reader:
        tweetCriteria = TweetCriteria().setQuerySearch(row['legal_name']).setMaxTweets(100)
        tweet = TweetManager.getTweets(tweetCriteria)
        data = pd.DataFrame([])
        try:
            for i in range(len(tweet)):
                #tweet_text = 
                #tweet_date = 
                #data = DataFrame({'Credit Union':row['legal_name'],'Tweet': TweetManager.getTweets(tweetCriteria)[i].text , 'Date': TweetManager.getTweets(tweetCriteria)[i].date}, index = [i])
                #data.append(DataFrame({'Credit Union':row['legal_name'],'Tweet': TweetManager.getTweets(tweetCriteria)[i].text , 'Date': TweetManager.getTweets(tweetCriteria)[i].date}, index = [i]))
                data = data.append(pd.DataFrame({'Credit Union':row['legal_name'],'Tweet': tweet[i].text , 'Date': tweet[i].date}, index=[i]), ignore_index=True)
                #print(data)
                #output[] = pd.concat([df, data],axis =0)
            df = df.append(data)
        except IndexError:
            continue
        continue
    
    #print(df.head())
    #print("appending :",output)
                
        #df.append(data)
    #df.append(df)           

In [20]:

df.to_csv("Prayash_segment#10.csv", sep=',', encoding='utf-8')

In [21]:

import pandas as pd
tweetCriteria = TweetCriteria().setQuerySearch("@pefcu").setMaxTweets(10)
data = pd.DataFrame([])
#for i in range(100):
                #tweet_text = 
                #tweet_date = 
                #data = DataFrame({'Credit Union':row['legal_name'],'Tweet': TweetManager.getTweets(tweetCriteria)[i].text , 'Date': TweetManager.getTweets(tweetCriteria)[i].date}, index = [i])
                #data.append(DataFrame({'Credit Union':row['legal_name'],'Tweet': TweetManager.getTweets(tweetCriteria)[i].text , 'Date': TweetManager.getTweets(tweetCriteria)[i].date}, index = [i]))
data = data.append(pd.DataFrame({'Credit Union':"Self-Help Credit Union",'Tweet': TweetManager.getTweets(tweetCriteria)[i].text , 'Date': TweetManager.getTweets(tweetCriteria)[i].date}, index=[i]), ignore_index=True)
print(data)
                #output[] = pd.concat([df, data],axis =0)
#df = df.append(data)

IndexError: list index out of range